# Few-shot JERX task offline inference

In [ ]:
#|default_exp jerx.offline.llm

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
from pathlib import Path
import pandas as pd
from bellek.jerx.utils import parse_triplets
from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# |export


def parse_triplet_response(response: str, *args, **kwargs) -> list[tuple[str, str, str]]:
    triplets = parse_triplets(response.strip())
    return [(e1, rel, e2) if e1 != e2 else (e1, rel, e2 + "(obj)") for e1, rel, e2 in triplets]


def make_kg_triplet_extract_fn(inference_cache_filepath: Path):
    df = pd.read_json(inference_cache_filepath, orient='records', lines=True)
    mapping = {text: generation for text, generation in zip(df["text"], df["generation"])}
    del df

    def extract_kg_triplets(text: str) -> list[tuple[str, str, str]]:
        return parse_triplet_response(mapping[text])

    return extract_kg_triplets

In [ ]:
#|hide
jerx_inferences_filepath = Path("../data/raw/musique-evaluation/jerx-inferences/llama3-base.jsonl")

df = pd.read_json(jerx_inferences_filepath, orient='records', lines=True)

df

,id,paragraph_idx,is_supporting,text,__index_level_0__,input,generation
0,2hop__131818_161450,1,True,# Golestan Province\nGolestān Province (Persia...,0,[{'content': 'You are an excellent knowledge g...,Golestan Province | location | north-east of I...
1,2hop__131818_161450,2,True,# Voshmgir District\nVoshmgir District () is a...,1,[{'content': 'You are an excellent knowledge g...,"Voshmgir District | location | Aqqala County, ..."
2,2hop__444265_82341,0,True,"# Ocala, Florida\nOcala (/ oʊ ˈkælə / oh - KAL...",2,[{'content': 'You are an excellent knowledge g...,Ocala | location | Florida\nOcala | location t...
3,2hop__444265_82341,12,True,"# Tom Denney\nTom Denney (born November 23, 19...",3,[{'content': 'You are an excellent knowledge g...,"Tom Denney | born on | November 23, 1982\nTom ..."
4,2hop__711946_269414,3,True,# All Your Faded Things\nAll Your Faded Things...,4,[{'content': 'You are an excellent knowledge g...,All Your Faded Things | album title | \nAll Yo...
...,...,...,...,...,...,...,...
1593,2hop__22431_16335,18,True,"# Switzerland\nIn 2012, resident foreigners ma...",2497,[{'content': 'You are an excellent knowledge g...,Switzerland | percentage of population | 23.3 ...
1594,2hop__132469_533882,17,True,# AAM-N-4 Oriole\nThe AAM-N-4 Oriole was an ea...,2499,[{'content': 'You are an excellent knowledge g...,AAM-N-4 Oriole | developer | Glenn L. Martin C...
1595,2hop__68981_39234,16,True,"# Tajikistan\nThe Kushan Empire, a collection ...",2500,[{'content': 'You are an excellent knowledge g...,Tajikistan | location | Central Asia\nKushan E...
1596,2hop__68981_39234,17,True,# Religion in Asia\nCountry Population Christi...,2501,[{'content': 'You are an excellent knowledge g...,"Brunei | population | 400,000\nBrunei | Christ..."


In [ ]:
#|hide
func = make_kg_triplet_extract_fn(jerx_inferences_filepath)
text = df.iloc[0]['text']
print(func(text))

[('Golestan Province', 'location', 'north-east of Iran'), ('Golestan Province', 'location relative to Caspian Sea', 'south'), ('Golestan Province', 'capital', 'Gorgan')]


In [ ]:
row = df.iloc[0]
row

id                                                 2hop__131818_161450
paragraph_idx                                                        1
is_supporting                                                     True
text                 # Golestan Province\nGolestān Province (Persia...
__index_level_0__                                                    0
input                [{'content': 'You are an excellent knowledge g...
generation           Golestan Province | location | north-east of I...
Name: 0, dtype: object

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()